# Tutorial 2: Interacting with Databases in Sessions

In this tutorial we will create and then interact with a *relational* SQLite database, using `sqlalchemy`'s `Session` object.

We will:

* Create a relational database containing two linked tables: "Objects" (with ID, ra and dec) and "Distances" (with ID and distance).
* Retrieve data into numpy arrays so that it can be used For Science.

This tutorial was adapted for astronomy from [the one at `pythoncentral.io`](http://pythoncentral.io/introductory-tutorial-python-sqlalchemy/).

### Requirements

You will need to have `sqlalchemy` installed.
```
pip install sqlalchemy
```

In [8]:
import sqlalchemy as sq
import os

In the previous tutorial, we used a somewhat manual approach where we separately defined a Table in a database, a mapper that would connect a python object to the database, and the python object itself. 

In this tutorial, we will use declaratives instead. The advantage of using declaratives is that instead of writing code for `Table`, mappers, and python class objects at different places, they can be defined at once in a class definition.

In [9]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship

## Creating Database

In [169]:
Base = declarative_base()

In [170]:
class Object(Base):
    __tablename__ = 'object'
    
    #Define columns like we did in the last notebook
    id = sq.Column(sq.Integer, primary_key=True)
    ra = sq.Column(sq.Float, nullable=False)
    dec = sq.Column(sq.Float, nullable=False)

    
class Distance(Base):
    __tablename__ = 'distance'
    
    #Again define columns like above. These columns
    #are normal python instance attributes.
    id = sq.Column(sq.Integer, primary_key=True)
    object_id = sq.Column(sq.Integer, sq.ForeignKey("object.id"), nullable=False)
    distance = sq.Column(sq.Float, nullable=False)

In [171]:
dbfile = 'sessions.db'

try: os.remove(dbfile)
except: pass

In [172]:
engine = sq.create_engine('sqlite:///'+dbfile)

In [173]:
Base.metadata.create_all(engine)

In [174]:
!wc -c $dbfile

   12288 sessions.db


## Inserting Data

In [175]:
from sqlalchemy.orm import sessionmaker

In [176]:
Base.metadata.bind = engine

In [177]:
DBSession = sessionmaker(bind=engine)

In [178]:
session = DBSession()

In [179]:
new_object = Object(ra=10.23, dec=-32.36)
session.add(new_object)
session.commit()

In [192]:
for i in range(100):
    new_distance = Distance(distance=4, object_id=1)
    session.add(new_distance)
    session.commit()

## Querying Data

In [190]:
obj = session.query(Object).first()
obj.id, obj.ra, obj.dec

(1, 10.23, -32.36)

In [196]:
dist = session.query(Distance)
distances = [d.]

101